In [2]:
import tkinter as tk
from tkinter import filedialog, colorchooser
from PIL import Image, ImageTk, ImageFilter, ImageEnhance
from tkinter import ttk  # Import themed Tkinter for modern styling

class PhotoEditorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Photo Editor")

        self.image_path = None
        self.original_image = None
        self.edited_image = None
        self.border_size = tk.IntVar()
        self.border_color = tk.StringVar()
        self.border_color.set("#000000")  # Default border color: Black
        self.filter_var = tk.StringVar()
        self.filter_var.set("Blur")  # Default filter: Blur

        # Set a modern theme
        self.style = ttk.Style()
        self.style.theme_use("clam")

        # Create widgets
        self.create_widgets()

    def create_widgets(self):
        # Open Image Button
        open_button = ttk.Button(self.root, text="Open Image", command=self.open_image)
        open_button.grid(row=0, column=0, padx=10, pady=10, sticky="ew")

        # Filter Dropdown
        filter_label = ttk.Label(self.root, text="Filter:")
        filter_label.grid(row=0, column=1, padx=10, pady=10, sticky="e")

        filter_options = ["Blur", "Black and White", "Sepia"]
        filter_dropdown = ttk.Combobox(self.root, textvariable=self.filter_var, values=filter_options)
        filter_dropdown.grid(row=0, column=2, padx=10, pady=10, sticky="ew")

        # Apply Filter Button
        apply_filter_button = ttk.Button(self.root, text="Apply Filter", command=self.apply_filter)
        apply_filter_button.grid(row=0, column=3, padx=10, pady=10, sticky="ew")

        # Border Size Entry
        border_size_label = ttk.Label(self.root, text="Border Size:")
        border_size_label.grid(row=0, column=4, padx=10, pady=10, sticky="e")

        border_size_entry = ttk.Entry(self.root, textvariable=self.border_size)
        border_size_entry.grid(row=0, column=5, padx=10, pady=10, sticky="ew")
        self.border_size.set(20)  # Default border size

        # Border Color Picker
        border_color_label = ttk.Label(self.root, text="Border Color:")
        border_color_label.grid(row=0, column=6, padx=10, pady=10, sticky="e")

        border_color_button = ttk.Button(self.root, text="Pick Color", command=self.pick_border_color)
        border_color_button.grid(row=0, column=7, padx=10, pady=10, sticky="ew")

        # Add Border Button
        add_border_button = ttk.Button(self.root, text="Add Border", command=self.add_border)
        add_border_button.grid(row=0, column=8, padx=10, pady=10, sticky="ew")

        # Clear All Filters Button
        clear_filters_button = ttk.Button(self.root, text="Clear All Filters", command=self.clear_filters)
        clear_filters_button.grid(row=0, column=9, padx=10, pady=10, sticky="ew")

        # Save Image Button
        save_button = ttk.Button(self.root, text="Save Image", command=self.save_image)
        save_button.grid(row=0, column=10, padx=10, pady=10, sticky="ew")

        # Canvas for displaying the image
        self.canvas = tk.Canvas(self.root, width=500, height=500)
        self.canvas.grid(row=1, column=0, columnspan=11, padx=10, pady=10, sticky="nsew")

        # Configure column and row weights for resizing
        for i in range(11):
            self.root.columnconfigure(i, weight=1)
        self.root.rowconfigure(1, weight=1)

    def open_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.gif")])

        if file_path:
            self.image_path = file_path
            self.original_image = Image.open(self.image_path)
            self.edited_image = self.original_image.copy()
            self.display_image()

    def apply_filter(self):
        if self.edited_image:
            selected_filter = self.filter_var.get()

            if selected_filter == "Blur":
                self.edited_image = self.edited_image.filter(ImageFilter.BLUR)
            elif selected_filter == "Black and White":
                self.edited_image = ImageEnhance.Color(self.edited_image).enhance(0.0)
            elif selected_filter == "Sepia":
                self.edited_image = self.apply_sepia_filter(self.edited_image)

            self.display_image()

    def apply_sepia_filter(self, image):
        sepia = Image.new('RGB', image.size, (112, 66, 20))
        sepia = Image.blend(image, sepia, 0.5)
        return sepia

    def add_border(self):
        if self.edited_image:
            border_size = self.border_size.get()
            border_color = self.border_color.get()

            # Add a border with the specified size and color
            new_width = self.edited_image.width + 2 * border_size
            new_height = self.edited_image.height + 2 * border_size

            bordered_image = Image.new("RGB", (new_width, new_height), border_color)
            bordered_image.paste(self.edited_image, (border_size, border_size))

            self.edited_image = bordered_image
            self.display_image()

    def pick_border_color(self):
        color = colorchooser.askcolor(title="Pick a Border Color", initialcolor=self.border_color.get())[1]
        if color:
            self.border_color.set(color)

    def clear_filters(self):
        self.edited_image = self.original_image.copy()
        self.display_image()

    def save_image(self):
        if self.edited_image:
            save_path = filedialog.asksaveasfilename(defaultextension=".png",
                                                       filetypes=[("PNG files", "*.png"),
                                                                  ("JPEG files", "*.jpg;*.jpeg")])
            if save_path:
                self.edited_image.save(save_path)

    def display_image(self):
        if self.edited_image:
            # Resize image to fit the canvas
            resized_image = self.edited_image.resize((500, 500), Image.ANTIALIAS)
            photo = ImageTk.PhotoImage(resized_image)

            # Update canvas with the new image
            self.canvas.config(width=photo.width(), height=photo.height())
            self.canvas.create_image(0, 0, anchor=tk.NW, image=photo)
            self.canvas.image = photo

if __name__ == "__main__":
    root = tk.Tk()
    app = PhotoEditorApp(root)
    root.mainloop()
